# QTN Data Integration with PySpedas

This notebook demonstrates how to download and work with PSP QTN (Quasi-Thermal Noise) data using pyspedas.
QTN provides the most reliable density measurement from electric field instruments.
We'll use the same date range as the WIND MFI test: 2022/06/01 20:00:00.000 to 2022/06/02 02:00:00.000


In [ ]:
# Cell 1: Download QTN data and determine file paths
import pyspedas
import os
import cdflib

# Define the same date range as WIND MFI test
trange = ['2022/06/01 20:00:00.000', '2022/06/02 02:00:00.000']
qtn_datatype = 'sqtn_rfs_V1V2'  # QTN data for most reliable density measurement

print(f"Downloading PSP QTN data for time range: {trange}")
print(f"Datatype: {qtn_datatype}")

# Download with downloadonly=True and notplot=True
downloaded_files = pyspedas.psp.fields(
    trange=trange, 
    datatype=qtn_datatype, 
    level='l3', 
    time_clip=True,
    get_support_data=True,
    downloadonly=True,  # Only download, don't load into memory
    notplot=True        # Don't create plots
)

print(f"\nDownload completed. Files returned: {len(downloaded_files) if downloaded_files else 0}")

if downloaded_files:
    for i, file_path in enumerate(downloaded_files):
        print(f"File {i+1}: {file_path}")
        
        # Get absolute path
        abs_path = os.path.abspath(file_path)
        print(f"  Absolute path: {abs_path}")
        
        # Check if file exists
        if os.path.exists(abs_path):
            file_size = os.path.getsize(abs_path) / (1024*1024)  # MB
            print(f"  File size: {file_size:.2f} MB")
            print(f"  File exists: Yes")
        else:
            print(f"  File exists: No")
        
        # Show directory structure
        directory = os.path.dirname(abs_path)
        print(f"  Directory: {directory}")
        
        print()
else:
    print("No files were downloaded or found.")


In [ ]:
# Cell 2: Extract variable names from the CDF file
import cdflib

if downloaded_files and len(downloaded_files) > 0:
    # Use the first downloaded file
    cdf_file_path = downloaded_files[0]
    abs_cdf_path = os.path.abspath(cdf_file_path)
    
    print(f"Analyzing CDF file: {os.path.basename(abs_cdf_path)}")
    print(f"Full path: {abs_cdf_path}")
    print("="*80)
    
    try:
        # Open the CDF file
        with cdflib.CDF(abs_cdf_path) as cdf:
            # Get CDF info
            cdf_info = cdf.cdf_info()
            
            print(f"CDF File Info:")
            print(f"  CDF Version: {cdf_info.Version}")
            print(f"  Encoding: {getattr(cdf_info, 'Encoding', 'Unknown')}")
            print(f"  Majority: {getattr(cdf_info, 'Majority', 'Unknown')}")
            print(f"  Number of rDimensions: {getattr(cdf_info, 'Num_rdim', 0)}")
            print(f"  rDimension sizes: {getattr(cdf_info, 'rDim_sizes', [])}")
            print(f"  Number of zVariables: {len(cdf_info.zVariables)}")
            print(f"  Number of rVariables: {len(cdf_info.rVariables)}")
            print(f"  Compressed: {getattr(cdf_info, 'Compressed', 'Unknown')}")
            print(f"  Checksum: {getattr(cdf_info, 'Checksum', 'Unknown')}")
            print()
            
            # List all zVariables (most data variables)
            print("zVariables (data variables):")
            for i, var_name in enumerate(cdf_info.zVariables):
                try:
                    var_info = cdf.varinq(var_name)
                    print(f"  {i+1:2d}. {var_name:30s} - {var_info.Data_Type_Description:15s} - Shape: {var_info.Dim_Sizes}")
                except Exception as e:
                    print(f"  {i+1:2d}. {var_name:30s} - Error getting info: {e}")
            
            print()
            
            # List all rVariables (usually metadata)
            if cdf_info.rVariables:
                print("rVariables (metadata variables):")
                for i, var_name in enumerate(cdf_info.rVariables):
                    try:
                        var_info = cdf.varinq(var_name)
                        print(f"  {i+1:2d}. {var_name:30s} - {var_info.Data_Type_Description:15s} - Shape: {var_info.Dim_Sizes}")
                    except Exception as e:
                        print(f"  {i+1:2d}. {var_name:30s} - Error getting info: {e}")
            else:
                print("No rVariables found.")
            
            print()
            
            # Look specifically for QTN/density-related variables
            print("QTN/Density-related variables (containing 'qtn', 'dens', 'n_elec', or 'density'):")
            qtn_vars = []
            all_vars = cdf_info.zVariables + cdf_info.rVariables
            
            for var_name in all_vars:
                lower_name = var_name.lower()
                if any(keyword in lower_name for keyword in ['qtn', 'dens', 'n_elec', 'density', 'electron_density', 'ne']):
                    qtn_vars.append(var_name)
                    try:
                        var_info = cdf.varinq(var_name)
                        print(f"  • {var_name:30s} - {var_info.Data_Type_Description:15s} - Shape: {var_info.Dim_Sizes}")
                    except Exception as e:
                        print(f"  • {var_name:30s} - Error getting info: {e}")
            
            if not qtn_vars:
                print("  No obvious QTN/density-related variables found.")
            
            print()
            
            # Look for time variables
            print("Time variables (containing 'epoch' or 'time'):")
            time_vars = []
            for var_name in all_vars:
                lower_name = var_name.lower()
                if 'epoch' in lower_name or 'time' in lower_name:
                    time_vars.append(var_name)
                    try:
                        var_info = cdf.varinq(var_name)
                        print(f"  • {var_name:30s} - {var_info.Data_Type_Description:15s} - Shape: {var_info.Dim_Sizes}")
                    except Exception as e:
                        print(f"  • {var_name:30s} - Error getting info: {e}")
            
            if not time_vars:
                print("  No time variables found.")
            
            print()
            print(f"Total variables found: {len(all_vars)}")
            print(f"QTN/density-related variables: {len(qtn_vars)}")
            print(f"Time variables: {len(time_vars)}")
            
    except Exception as e:
        print(f"Error reading CDF file: {e}")
        import traceback
        print(traceback.format_exc())
        
else:
    print("No CDF files available to analyze. Please run the download cell first.")


In [ ]:
# Cell 3: Examine electron density and QTN variables specifically
import numpy as np

if downloaded_files and len(downloaded_files) > 0:
    cdf_file_path = downloaded_files[0]
    abs_cdf_path = os.path.abspath(cdf_file_path)
    
    print(f"Examining electron density and QTN variables in: {os.path.basename(abs_cdf_path)}")
    print("="*80)
    
    try:
        with cdflib.CDF(abs_cdf_path) as cdf:
            # Get all variable names to search for relevant ones
            cdf_info = cdf.cdf_info()
            all_vars = cdf_info.zVariables + cdf_info.rVariables
            
            # Look for common QTN variable names
            potential_vars = ['n_elec', 'electron_density', 'density', 'DENS', 'ne', 'N_ELEC']
            
            found_vars = []
            for var in potential_vars:
                if var in all_vars:
                    found_vars.append(var)
            
            # If no exact matches, look for variables containing density-related keywords
            if not found_vars:
                print("No exact matches found, searching for variables containing density keywords...")
                for var_name in all_vars:
                    lower_name = var_name.lower()
                    if any(keyword in lower_name for keyword in ['dens', 'elec', 'ne', 'qtn']):
                        found_vars.append(var_name)
            
            if found_vars:
                for var_name in found_vars[:3]:  # Limit to first 3 variables to avoid too much output
                    print(f"{var_name} (Potential Electron Density/QTN Variable):")
                    try:
                        var_data = cdf.varget(var_name)
                        print(f"  Data type: {type(var_data)}")
                        print(f"  Array shape: {var_data.shape}")
                        print(f"  Data length: {len(var_data) if hasattr(var_data, '__len__') else 'N/A'}")
                        
                        if hasattr(var_data, 'dtype') and np.issubdtype(var_data.dtype, np.number):
                            print(f"  Min value: {np.nanmin(var_data):.6f}")
                            print(f"  Max value: {np.nanmax(var_data):.6f}")
                            print(f"  Mean value: {np.nanmean(var_data):.6f}")
                            print(f"  Number of valid (non-NaN) values: {np.sum(~np.isnan(var_data))}")
                            print(f"  Number of NaN values: {np.sum(np.isnan(var_data))}")
                            
                            # Show first few values if 1D array
                            if len(var_data.shape) == 1:
                                print(f"  First 10 values: {var_data[:10]}")
                            else:
                                print(f"  First value shape: {var_data[0].shape if len(var_data) > 0 else 'Empty'}")
                        else:
                            print(f"  Data type: {var_data.dtype if hasattr(var_data, 'dtype') else 'Unknown'}")
                            if hasattr(var_data, '__len__') and len(var_data) > 0:
                                print(f"  First few values: {var_data[:5]}")
                        
                        # Get variable attributes
                        try:
                            var_attrs = cdf.varattsget(var_name)
                            if "UNITS" in var_attrs:
                                print(f"  Units: {var_attrs['UNITS']}")
                            if "FIELDNAM" in var_attrs:
                                print(f"  Field name: {var_attrs['FIELDNAM']}")
                            if "CATDESC" in var_attrs:
                                print(f"  Description: {var_attrs['CATDESC']}")
                        except:
                            pass
                            
                    except Exception as e:
                        print(f"  Error reading {var_name}: {e}")
                    
                    print()
            
            else:
                print("No density or QTN-related variables found.")
                print("Available variables:")
                for var in all_vars[:10]:  # Show first 10 variables
                    print(f"  - {var}")
                if len(all_vars) > 10:
                    print(f"  ... and {len(all_vars) - 10} more variables")
            
            print()
            
            # Also check the time variable for context
            print("Epoch (Time variable for reference):")
            try:
                epoch_data = cdf.varget("Epoch")
                print(f"  Data type: {type(epoch_data)}")
                print(f"  Array shape: {epoch_data.shape}")
                print(f"  Data length: {len(epoch_data)}")
                print(f"  First timestamp: {cdflib.cdfepoch.to_datetime(epoch_data[0])}")
                print(f"  Last timestamp: {cdflib.cdfepoch.to_datetime(epoch_data[-1])}")
                print(f"  Total time span: {cdflib.cdfepoch.to_datetime(epoch_data[-1]) - cdflib.cdfepoch.to_datetime(epoch_data[0])}")
                
            except Exception as e:
                print(f"  Error reading Epoch: {e}")
                # Try alternative time variable names
                time_vars = [v for v in all_vars if 'time' in v.lower() or 'epoch' in v.lower()]
                if time_vars:
                    print(f"  Found alternative time variables: {time_vars}")
            
            print()
                
    except Exception as e:
        print(f"Error opening CDF file: {e}")
        import traceback
        print(traceback.format_exc())
        
else:
    print("No CDF files available to analyze. Please run the download cell first.")


In [ ]:
# Cell 1: Download alpha data and determine file paths
import pyspedas
import os
import cdflib

# Define the same date range as WIND MFI test
trange = ['2022/06/01 20:00:00.000', '2022/06/02 02:00:00.000']
spi_sf0a_datatype = 'spi_sf0a_l3_mom'  # Alpha particle moments

print(f"Downloading PSP alpha data for time range: {trange}")
print(f"Datatype: {spi_sf0a_datatype}")

# Download with downloadonly=True and notplot=True
downloaded_files = pyspedas.psp.spi(
    trange=trange, 
    datatype=spi_sf0a_datatype, 
    level='l3', 
    time_clip=True,
    downloadonly=True,  # Only download, don't load into memory
    notplot=True        # Don't create plots
)

print(f"\nDownload completed. Files returned: {len(downloaded_files) if downloaded_files else 0}")

if downloaded_files:
    for i, file_path in enumerate(downloaded_files):
        print(f"File {i+1}: {file_path}")
        
        # Get absolute path
        abs_path = os.path.abspath(file_path)
        print(f"  Absolute path: {abs_path}")
        
        # Check if file exists
        if os.path.exists(abs_path):
            file_size = os.path.getsize(abs_path) / (1024*1024)  # MB
            print(f"  File size: {file_size:.2f} MB")
            print(f"  File exists: Yes")
        else:
            print(f"  File exists: No")
        
        # Show directory structure
        directory = os.path.dirname(abs_path)
        print(f"  Directory: {directory}")
        
        print()
else:
    print("No files were downloaded or found.")


In [ ]:
# Cell 2: Extract variable names from the CDF file
import cdflib

if downloaded_files and len(downloaded_files) > 0:
    # Use the first downloaded file
    cdf_file_path = downloaded_files[0]
    abs_cdf_path = os.path.abspath(cdf_file_path)
    
    print(f"Analyzing CDF file: {os.path.basename(abs_cdf_path)}")
    print(f"Full path: {abs_cdf_path}")
    print("="*80)
    
    try:
        # Open the CDF file
        with cdflib.CDF(abs_cdf_path) as cdf:
            # Get CDF info
            cdf_info = cdf.cdf_info()
            
            print(f"CDF File Info:")
            print(f"  CDF Version: {cdf_info.Version}")
            print(f"  Encoding: {getattr(cdf_info, 'Encoding', 'Unknown')}")
            print(f"  Majority: {getattr(cdf_info, 'Majority', 'Unknown')}")
            print(f"  Number of rDimensions: {getattr(cdf_info, 'Num_rdim', 0)}")
            print(f"  rDimension sizes: {getattr(cdf_info, 'rDim_sizes', [])}")
            print(f"  Number of zVariables: {len(cdf_info.zVariables)}")
            print(f"  Number of rVariables: {len(cdf_info.rVariables)}")
            print(f"  Compressed: {getattr(cdf_info, 'Compressed', 'Unknown')}")
            print(f"  Checksum: {getattr(cdf_info, 'Checksum', 'Unknown')}")
            print()
            
            # List all zVariables (most data variables)
            print("zVariables (data variables):")
            for i, var_name in enumerate(cdf_info.zVariables):
                try:
                    var_info = cdf.varinq(var_name)
                    print(f"  {i+1:2d}. {var_name:30s} - {var_info.Data_Type_Description:15s} - Shape: {var_info.Dim_Sizes}")
                except Exception as e:
                    print(f"  {i+1:2d}. {var_name:30s} - Error getting info: {e}")
            
            print()
            
            # List all rVariables (usually metadata)
            if cdf_info.rVariables:
                print("rVariables (metadata variables):")
                for i, var_name in enumerate(cdf_info.rVariables):
                    try:
                        var_info = cdf.varinq(var_name)
                        print(f"  {i+1:2d}. {var_name:30s} - {var_info.Data_Type_Description:15s} - Shape: {var_info.Dim_Sizes}")
                    except Exception as e:
                        print(f"  {i+1:2d}. {var_name:30s} - Error getting info: {e}")
            else:
                print("No rVariables found.")
            
            print()
            
            # Look specifically for alpha-related variables
            print("Alpha-related variables (containing 'alpha', 'na', or 'va'):")
            alpha_vars = []
            all_vars = cdf_info.zVariables + cdf_info.rVariables
            
            for var_name in all_vars:
                lower_name = var_name.lower()
                if any(keyword in lower_name for keyword in ['alpha', 'na', 'va', 'temp_alpha', 'vel_alpha']):
                    alpha_vars.append(var_name)
                    try:
                        var_info = cdf.varinq(var_name)
                        print(f"  • {var_name:30s} - {var_info.Data_Type_Description:15s} - Shape: {var_info.Dim_Sizes}")
                    except Exception as e:
                        print(f"  • {var_name:30s} - Error getting info: {e}")
            
            if not alpha_vars:
                print("  No obvious alpha-related variables found.")
            
            print()
            
            # Look for time variables
            print("Time variables (containing 'epoch' or 'time'):")
            time_vars = []
            for var_name in all_vars:
                lower_name = var_name.lower()
                if 'epoch' in lower_name or 'time' in lower_name:
                    time_vars.append(var_name)
                    try:
                        var_info = cdf.varinq(var_name)
                        print(f"  • {var_name:30s} - {var_info.Data_Type_Description:15s} - Shape: {var_info.Dim_Sizes}")
                    except Exception as e:
                        print(f"  • {var_name:30s} - Error getting info: {e}")
            
            if not time_vars:
                print("  No time variables found.")
            
            print()
            print(f"Total variables found: {len(all_vars)}")
            print(f"Alpha-related variables: {len(alpha_vars)}")
            print(f"Time variables: {len(time_vars)}")
            
    except Exception as e:
        print(f"Error reading CDF file: {e}")
        import traceback
        print(traceback.format_exc())
        
else:
    print("No CDF files available to analyze. Please run the download cell first.")


# Alpha Particle Data Integration with PySpedas

This notebook demonstrates how to download and work with PSP alpha particle data using pyspedas.
We'll use the same date range as the WIND MFI test: 2022/06/01 20:00:00.000 to 2022/06/02 02:00:00.000


In [ ]:
# Cell 1: Download alpha data and determine file paths
import pyspedas
import os
import cdflib

# Define the same date range as WIND MFI test
trange = ['2022/06/01 20:00:00.000', '2022/06/02 02:00:00.000']
spi_sf0a_datatype = 'spi_sf0a_l3_mom'  # Alpha particle moments

print(f"Downloading PSP alpha data for time range: {trange}")
print(f"Datatype: {spi_sf0a_datatype}")

# Download with downloadonly=True and notplot=True
downloaded_files = pyspedas.psp.spi(
    trange=trange, 
    datatype=spi_sf0a_datatype, 
    level='l3', 
    time_clip=True,
    downloadonly=True,  # Only download, don't load into memory
    notplot=True        # Don't create plots
)

print(f"\nDownload completed. Files returned: {len(downloaded_files) if downloaded_files else 0}")

if downloaded_files:
    for i, file_path in enumerate(downloaded_files):
        print(f"File {i+1}: {file_path}")
        
        # Get absolute path
        abs_path = os.path.abspath(file_path)
        print(f"  Absolute path: {abs_path}")
        
        # Check if file exists
        if os.path.exists(abs_path):
            file_size = os.path.getsize(abs_path) / (1024*1024)  # MB
            print(f"  File size: {file_size:.2f} MB")
            print(f"  File exists: Yes")
        else:
            print(f"  File exists: No")
        
        # Show directory structure
        directory = os.path.dirname(abs_path)
        print(f"  Directory: {directory}")
        
        print()
else:
    print("No files were downloaded or found.")


In [ ]:
# Cell 2: Extract variable names from the CDF file
import cdflib

if downloaded_files and len(downloaded_files) > 0:
    # Use the first downloaded file
    cdf_file_path = downloaded_files[0]
    abs_cdf_path = os.path.abspath(cdf_file_path)
    
    print(f"Analyzing CDF file: {os.path.basename(abs_cdf_path)}")
    print(f"Full path: {abs_cdf_path}")
    print("="*80)
    
    try:
        # Open the CDF file
        with cdflib.CDF(abs_cdf_path) as cdf:
            # Get CDF info
            cdf_info = cdf.cdf_info()
            
            print(f"CDF File Info:")
            print(f"  CDF Version: {cdf_info.version}")
            print(f"  Number of dimensions: {cdf_info.num_dims}")
            print(f"  Number of zVariables: {len(cdf_info.zVariables)}")
            print(f"  Number of rVariables: {len(cdf_info.rVariables)}")
            print()
            
            # List all zVariables (most data variables)
            print("zVariables (data variables):")
            for i, var_name in enumerate(cdf_info.zVariables):
                try:
                    var_info = cdf.varinq(var_name)
                    print(f"  {i+1:2d}. {var_name:30s} - {var_info.Data_Type_Description:15s} - Shape: {var_info.Dim_Sizes}")
                except Exception as e:
                    print(f"  {i+1:2d}. {var_name:30s} - Error getting info: {e}")
            
            print()
            
            # List all rVariables (usually metadata)
            if cdf_info.rVariables:
                print("rVariables (metadata variables):")
                for i, var_name in enumerate(cdf_info.rVariables):
                    try:
                        var_info = cdf.varinq(var_name)
                        print(f"  {i+1:2d}. {var_name:30s} - {var_info.Data_Type_Description:15s} - Shape: {var_info.Dim_Sizes}")
                    except Exception as e:
                        print(f"  {i+1:2d}. {var_name:30s} - Error getting info: {e}")
            else:
                print("No rVariables found.")
            
            print()
            
            # Look specifically for alpha-related variables
            print("Alpha-related variables (containing 'alpha', 'na', or 'va'):")
            alpha_vars = []
            all_vars = cdf_info.zVariables + cdf_info.rVariables
            
            for var_name in all_vars:
                lower_name = var_name.lower()
                if any(keyword in lower_name for keyword in ['alpha', 'na', 'va', 'temp_alpha', 'vel_alpha']):
                    alpha_vars.append(var_name)
                    try:
                        var_info = cdf.varinq(var_name)
                        print(f"  • {var_name:30s} - {var_info.Data_Type_Description:15s} - Shape: {var_info.Dim_Sizes}")
                    except Exception as e:
                        print(f"  • {var_name:30s} - Error getting info: {e}")
            
            if not alpha_vars:
                print("  No obvious alpha-related variables found.")
            
            print()
            
            # Look for time variables
            print("Time variables (containing 'epoch' or 'time'):")
            time_vars = []
            for var_name in all_vars:
                lower_name = var_name.lower()
                if 'epoch' in lower_name or 'time' in lower_name:
                    time_vars.append(var_name)
                    try:
                        var_info = cdf.varinq(var_name)
                        print(f"  • {var_name:30s} - {var_info.Data_Type_Description:15s} - Shape: {var_info.Dim_Sizes}")
                    except Exception as e:
                        print(f"  • {var_name:30s} - Error getting info: {e}")
            
            if not time_vars:
                print("  No time variables found.")
            
            print()
            print(f"Total variables found: {len(all_vars)}")
            print(f"Alpha-related variables: {len(alpha_vars)}")
            print(f"Time variables: {len(time_vars)}")
            
    except Exception as e:
        print(f"Error reading CDF file: {e}")
        import traceback
        print(traceback.format_exc())
        
else:
    print("No CDF files available to analyze. Please run the download cell first.")
